### Librerías

In [1]:
import time
import random
import pandas as pd
from datetime import datetime

### Datos de entrada y ruta de salida

In [2]:
# Lectura de los archivos CSV con información de ligas y equipos
raw_leagues_df = pd.read_csv('../data/df_ligas.csv')     # Datos de ligas y sus URLs
team_table_df = pd.read_csv('../data/team_table.csv')    # Datos de la tabla de equipos

# Definición de la ruta de salida para el archivo procesado
output_path = '../data/team_positions_leagues.csv'


### Función que obtiene la información de las ligas

In [3]:
def get_leagues_data(url: str, liga: str) -> pd.DataFrame:
    """
    Obtiene datos de una liga desde una URL, hace limpieza inicial y añade metadatos.
    """

    # Pausa aleatoria para evitar bloqueos al hacer scraping (1, 3 o 2 segundos)
    tiempo = [1, 3, 2]
    time.sleep(random.choice(tiempo))

    # Lee todas las tablas HTML de la URL
    df = pd.read_html(url)

    # Combina las dos primeras tablas en un solo DataFrame
    df = pd.concat([df[0], df[1]], ignore_index=True, axis=1)

    # Renombra las columnas con nombres significativos
    df = df.rename(
        columns={
            0: 'EQUIPO', 1: 'J', 2: 'G', 3: 'E', 4: 'P',
            5: 'GF', 6: 'GC', 7: 'DIF', 8: 'PTS'
        }
    )

    # Limpia el nombre del equipo, eliminando índices o caracteres iniciales
    df['EQUIPO'] = df['EQUIPO'].apply(
        lambda x: x[5:] if x[:2].isnumeric() else x[4:]
    )

    # Agrega el nombre de la liga como columna
    df['LIGA'] = liga

    # Agrega la fecha de ejecución como metadato
    run_date = datetime.now().strftime("%Y-%m-%d")
    df['CREATED_AT'] = run_date

    # Devuelve el DataFrame limpio y enriquecido
    return df

### Función que une la información de cada liga

In [4]:
def get_all_leagues_data(raw_leagues_df: pd.DataFrame) -> pd.DataFrame:
    """
    Itera sobre las ligas y obtiene datos concatenados en un solo DataFrame.
    """

    # Lista para almacenar los datos de cada liga
    all_leagues = []

    # Itera sobre cada fila del DataFrame de ligas
    for _, row in raw_leagues_df.iterrows():
        # Llama a get_leagues_data para obtener los datos de la liga actual
        league_df = get_leagues_data(row['URL'], row['LIGA'])
        # Agrega el DataFrame resultante a la lista
        all_leagues.append(league_df)

    # Concatena todos los DataFrames en uno solo
    all_leagues_df = pd.concat(all_leagues, ignore_index=True)

    # Devuelve el DataFrame combinado
    return all_leagues_df


In [5]:
get_all_leagues_data(raw_leagues_df)

,EQUIPO,J,G,E,P,GF,GC,DIF,PTS,LIGA,CREATED_AT
0,Manchester City,1,1,0,0,4,0,4,3,INGLATERRA,2025-08-22
1,Sunderland,1,1,0,0,3,0,3,3,INGLATERRA,2025-08-22
2,Tottenham Hotspur,1,1,0,0,3,0,3,3,INGLATERRA,2025-08-22
3,Liverpool,1,1,0,0,4,2,2,3,INGLATERRA,2025-08-22
4,Nottingham Forest,1,1,0,0,3,1,2,3,INGLATERRA,2025-08-22
...,...,...,...,...,...,...,...,...,...,...,...
127,C.D. Nacional,2,0,1,1,1,3,-2,1,PORTUGAL,2025-08-22
128,AVS,2,0,0,2,1,5,-4,0,PORTUGAL,2025-08-22
129,Alverca,2,0,0,2,1,5,-4,0,PORTUGAL,2025-08-22
130,Santa Clara,2,0,0,2,0,4,-4,0,PORTUGAL,2025-08-22


### Función que une los datos de ligas y la tabla de equipos

In [6]:
def merge_with_teams(all_leagues_df: pd.DataFrame, team_table_df: pd.DataFrame) -> pd.DataFrame:
    """
    Une los datos de ligas con la tabla de equipos.
    """

    # Realiza un merge interno entre los datos de ligas y la tabla de equipos usando la columna 'EQUIPO'
    merge_df = pd.merge(all_leagues_df, team_table_df, how='inner', on='EQUIPO')

    # Reordena y selecciona las columnas relevantes para el resultado final
    merge_df = merge_df[
        ['ID_TEAM', 'EQUIPO', 'J', 'G', 'E', 'P', 'GF', 'GC', 'DIF', 'PTS', 'LIGA', 'CREATED_AT']
    ]

    # Devuelve el DataFrame resultante con los datos consolidados
    return merge_df


### Función final

In [7]:
def data_processing(raw_leagues_df: pd.DataFrame, team_table_df: pd.DataFrame, output_path: str = None) -> pd.DataFrame:
    """
    Función envoltura que ejecuta todo el flujo: 
    1) obtiene datos de ligas 
    2) los une con tabla de equipos 
    3) guarda CSV opcionalmente.
    """

    # Obtiene los datos de todas las ligas desde las URLs
    all_leagues_df = get_all_leagues_data(raw_leagues_df)

    # Combina los datos de ligas con la tabla de equipos
    final_df = merge_with_teams(all_leagues_df, team_table_df)

    # Si se proporciona una ruta de salida, guarda el resultado en un CSV
    if output_path:
        final_df.to_csv(output_path, index=False)

    # Devuelve el DataFrame procesado
    return final_df


In [9]:
data_processing(raw_leagues_df,team_table_df, output_path)

,ID_TEAM,EQUIPO,J,G,E,P,GF,GC,DIF,PTS,LIGA,CREATED_AT
0,ffd4f419,Manchester City,1,1,0,0,4,0,4,3,INGLATERRA,2025-08-22
1,ffd4f41a,Tottenham Hotspur,1,1,0,0,3,0,3,3,INGLATERRA,2025-08-22
2,ffd4f41f,Liverpool,1,1,0,0,4,2,2,3,INGLATERRA,2025-08-22
3,ffd4f42a,Nottingham Forest,1,1,0,0,3,1,2,3,INGLATERRA,2025-08-22
4,ffd4f418,Arsenal,1,1,0,0,1,0,1,3,INGLATERRA,2025-08-22
...,...,...,...,...,...,...,...,...,...,...,...,...
83,ffd4f45c,Guimaraes,2,1,0,1,3,5,-2,3,PORTUGAL,2025-08-22
84,ffd4f45f,Arouca,2,1,0,1,3,7,-4,3,PORTUGAL,2025-08-22
85,ffd4f460,Rio Ave,1,0,1,0,1,1,0,1,PORTUGAL,2025-08-22
86,ffd4f45a,Estoril,2,0,1,1,3,4,-1,1,PORTUGAL,2025-08-22


Extracción -> Procesamiento -> Transformación -> Carga 